<a href="https://colab.research.google.com/github/hfelizzola/Investigaciones-de-Operaciones-I/blob/main/01_problema_produccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Tutorial: Modelo de Producción con GAMSPy

### Ejemplo 1: Producción en un Solo Periodo (Taha, 2012)

Para la temporada de fin de año, una fábrica de ropa está produciendo abrigos con capota, chaquetas de plumas, pantalones térmicos y guantes. La producción pasa por cuatro departamentos: corte, aislamiento, costura y empaque. La fábrica ya tiene pedidos fijos de sus clientes. El contrato establece una multa (penalización) por cada prenda no entregada. El objetivo es crear un plan de producción que **maximice la ganancia**.

---

### 🧮 Datos de Producción y Costos

| Departamento   | Abrigos | Chaquetas de Plumas | Pantalones Térmicos | Guantes | Capacidad (h) |
|----------------|---------|---------------------|----------------------|---------|----------------|
| Corte          | 0.30    | 0.30                | 0.25                 | 0.15    | 1000           |
| Aislamiento    | 0.25    | 0.35                | 0.30                 | 0.10    | 1000           |
| Costura        | 0.45    | 0.50                | 0.40                 | 0.22    | 1000           |
| Empaque        | 0.15    | 0.15                | 0.10                 | 0.05    | 1000           |
| **Demanda**    | 800     | 750                 | 600                  | 500     |                |
| **Utilidad (\$)** | 30      | 40                  | 20                   | 10      |                |
| **Penalización (\$)** | 15      | 20                  | 10                   | 8       |                |

---

### 📐 Formulación del Modelo

#### 1. Conjuntos (Sets)

- $i$: Departamentos, $j \in \{1:\text{Corte}, 2:\text{Aislamiento}, 3:\text{Costura}, 4:\text{Empaque}\}$
- $j$: Productos, $j \in \{1:\text{Abrigos}, 2:\text{Chaquetas}, 3:\text{Pantalones}, 4:\text{Guantes}\}$

#### 2. Variables de Decisión

-  $x_j$: Cantidad a fabricar del producto $j$, donde  
- $ s_j$: Faltantes (unidades no entregadas) del producto $j$

#### 3. Parámetros

- $CA_i$: Capacidad en horas de cada departamento $i$
- $DE_i$: Demanda de cada producto $i$
- $UT_i$: Utilidad de cada producto $i$
- $PE_i$: Demanda por cada faltante de producto $i$
- $TP_{ij}$: Tiempo de producto del producto $i$ en el departamento $j$

#### Función Objetivo

Maximizar la utilidad total (ganancia por ventas menos multa por faltantes):


\begin{aligned}
\text{Max. } Z = (30x_1 + 40x_2 + 20x_3 + 10x_4) - (15s_1 + 20s_2 + 10s_3 + 8s_4)
\end{aligned}

#### Restricciones

1. **Capacidad de Departamentos**  
   El tiempo total usado en cada departamento no puede superar su capacidad:


\begin{aligned}
0.30x_1 + 0.30x_2 + 0.25x_3 + 0.15x_4 &\le 1000 \quad \text{(Corte)} \\
0.25x_1 + 0.35x_2 + 0.30x_3 + 0.10x_4 &\le 1000 \quad \text{(Aislamiento)} \\
0.45x_1 + 0.50x_2 + 0.40x_3 + 0.22x_4 &\le 1000 \quad \text{(Costura)} \\
0.15x_1 + 0.15x_2 + 0.10x_3 + 0.05x_4 &\le 1000 \quad \text{(Empaque)}
\end{aligned}


2. **Balance de Demanda**  
   Lo fabricado más lo que falta por entregar debe ser igual a la demanda:


\begin{aligned}
x_1 + s_1 &= 800 \quad \text{(Abrigos)} \\
x_2 + s_2 &= 750 \quad \text{(Chaquetas)} \\
x_3 + s_3 &= 600 \quad \text{(Pantalones)} \\
x_4 + s_4 &= 500 \quad \text{(Guantes)}
\end{aligned}


3. **No Negatividad**


\begin{aligned}
x_j \ge 0, \quad s_j \ge 0, \quad \forall j = 1, 2, 3, 4
\end{aligned}


## Instalación de la librería GAMSPY

In [ ]:
# Instalación de gampspy sin mostrar progreso
!pip install gamspy --quiet

## Importar librerias

In [ ]:
# Importar librerías necesarias
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sense, Sum
import numpy as np
import pandas as pd

## Programación del Modelo

In [ ]:
# Paso 1: Crear contenedor del modelo
m = Container()

In [ ]:
# Paso 2: Definir conjuntos
productos = Set(m, name="productos", records=["abrigos", "chaquetas", "pantalones", "guantes"])
departamentos = Set(m, name="departamentos", records=["corte", "aislamiento", "costura", "empaque"])

In [ ]:
# Visualizar las variables
productos.records

,uni,element_text
0,abrigos,
1,chaquetas,
2,pantalones,
3,guantes,


In [ ]:
departamentos.records

,uni,element_text
0,corte,
1,aislamiento,
2,costura,
3,empaque,


In [ ]:
# Paso 3: Parámetros del modelo

# Tiempos por unidad en cada departamento (en horas)
tiempo = Parameter(m, name="tiempo", domain=[departamentos, productos], records=np.array([
   [0.30, 0.30, 0.25, 0.15],
   [0.25, 0.35, 0.30, 0.10],
   [0.45, 0.50, 0.40, 0.22],
   [0.15, 0.15, 0.10, 0.05]
]))


# Capacidad por departamento
capacidad = Parameter(m, name="capacidad", domain=[departamentos], records=np.array([1000, 1000, 1000, 1000]))

# Demanda por producto
demanda = Parameter(m, name="demanda", domain=[productos], records=np.array([800, 750, 600, 500]))

# Utilidad por unidad fabricada
utilidad = Parameter(m, name="utilidad", domain=[productos], records=np.array([30, 40, 20, 10]))

# Penalización por faltantes
penalizacion = Parameter(m, name="penalizacion", domain=[productos], records=np.array([15, 20, 10, 8]))

In [ ]:
tiempo.records

,departamentos,productos,value
0,corte,abrigos,0.30
1,corte,chaquetas,0.30
2,corte,pantalones,0.25
3,corte,guantes,0.15
4,aislamiento,abrigos,0.25
5,aislamiento,chaquetas,0.35
6,aislamiento,pantalones,0.30
7,aislamiento,guantes,0.10
8,costura,abrigos,0.45
9,costura,chaquetas,0.50


In [ ]:
capacidad.records

,departamentos,value
0,corte,1000.0
1,aislamiento,1000.0
2,costura,1000.0
3,empaque,1000.0


In [ ]:
# Paso 4: Variables de decisión
x = Variable(m, name="x", type="Positive", domain=[productos])  # Cantidad producida
s = Variable(m, name="s", type="Positive", domain=[productos])  # Faltantes

In [ ]:
x.records

In [ ]:
# Paso 5: Definir restricciones

# Restricción de capacidad por departamento
capacidad_uso = Equation(m, name="capacidad_uso", domain=[departamentos])
capacidad_uso[...] = Sum(productos, tiempo[departamentos, productos] * x[productos]) <= capacidad[departamentos]

# Balance de demanda por producto
balance_demanda = Equation(m, name="balance_demanda", domain=[productos])
balance_demanda[...] = x[productos] + s[productos] == demanda[productos]

In [ ]:
# Paso 6: Función objetivo
objetivo = Sum(productos, utilidad[productos] * x[productos] - penalizacion[productos] * s[productos])

In [ ]:
# Paso 7: Crear y resolver el modelo
modelo = Model(m,
               name="ProduccionRopa",
               equations=[capacidad_uso, balance_demanda],
               sense=Sense.MAX,
               objective=objetivo,
               problem="LP")
modelo.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,64625.0,9,9,LP,CPLEX,0.001


## Análisis de la solución

La siguiente nos muestra la solución de la variable $x$, lo que nos indica que el plan de producción es:

- **800 abrigos**
- **750 Chaquetas**
- **387.5 Pantalones**
- **500 Guantes**



In [ ]:
# Solución de la variable x
x.records

,productos,level,marginal,lower,upper,scale
0,abrigos,800.0,0.0,0.0,inf,1.0
1,chaquetas,750.0,0.0,0.0,inf,1.0
2,pantalones,387.5,0.0,0.0,inf,1.0
3,guantes,500.0,0.0,0.0,inf,1.0


Tambien podemos ver la solución de la variable $s$. En esta tabla se puede observar que:

- No se generan faltantes en los abrigos, chaquetas y guantes.
- Se generan 212.5 faltantes de pantalones.

In [ ]:
# Solución de la variable s
s.records

,productos,level,marginal,lower,upper,scale
0,abrigos,0.0,-11.25,0.0,inf,1.0
1,chaquetas,0.0,-22.50,0.0,inf,1.0
2,pantalones,212.5,0.00,0.0,inf,1.0
3,guantes,0.0,-1.50,0.0,inf,1.0


Con este plan de producción el la utilidad total es de **$64.625**

In [ ]:
# Valor objetivo: Utilidad Total
modelo.objective_value

64625.0

Tambien podemos hacer una análisis de la capacidad usada. De la siguiente tabla podemos concluir que:

- **Corte:** se utilizan 636,875 horas de las 1000 disponibles.
- **Aislamiento:** se utilizan 628,750 horas de las 1000 disponibles.
- **Costura:** se utilizan 1000 horas de las 1000 disponibles. Por tanto, es un recurso restrictivo.
- **Empaque:** se utilizan 296,250 horas de las 100 disponibles.

In [ ]:
capacidad_uso.records

,departamentos,level,marginal,lower,upper,scale
0,corte,636.875,0.0,-inf,1000.0,1.0
1,aislamiento,628.750,0.0,-inf,1000.0,1.0
2,costura,1000.000,75.0,-inf,1000.0,1.0
3,empaque,296.250,0.0,-inf,1000.0,1.0


## Modelo Explicito

In [ ]:
m = Container()

# Paso 1: Definir conjuntos (solo para organización, no se usan en las ecuaciones)
productos = Set(m, name="productos", records=[1, 2, 3, 4])
departamentos = Set(m, name="departamentos", records=[1, 2, 3, 4])

# Paso 2: Definir variables de decisión

# Variables de producción
x = Variable(m, name="x", domain=productos,type="Positive")

# Variables de faltantes
s = Variable(m, name="s", domain=productos, type="Positive")

# Paso 3: Función objetivo
objetivo = 30*x[1] + 40*x[2] + 20*x[3] + 10*x[4] - 15*s[1] - 20*s[2] - 10*s[3] - 8*s[4]

# Paso 4: Restricciones

# Restricciones de capacidad
# Restriccion 1 departamento de corte
restriccion_corte = Equation(m, name="restriccion_corte")
restriccion_corte[...] = 0.30*x[1] + 0.30*x[2] + 0.25*x[3] + 0.15*x[4] <= 1000

# Restricción 2 departamento de aislamiento
restriccion_aislamiento = Equation(m, name="restriccion_aislamiento")
restriccion_aislamiento[...] = 0.25*x[1] + 0.35*x[2] + 0.30*x[3] + 0.10*x[4] <= 1000

# Restricción 3 departamento de costura
restriccion_costura = Equation(m, name="restriccion_costura")
restriccion_costura[...] = 0.45*x[1] + 0.50*x[2] + 0.40*x[3] + 0.22*x[4] <= 1000

# Restricción 4 departamento de empaque
restriccion_empaque = Equation(m, name="restriccion_empaque")
restriccion_empaque[...] = 0.15*x[1] + 0.15*x[2] + 0.10*x[3] + 0.05*x[4] <= 1000

# Restricciones de demanda
# Balance de demanda de ABRIGOS
balance_abrigos = Equation(m, name="balance_abrigos")
balance_abrigos[...] = x[1] + s[1] == 800

# Balance de demanda de CHAQUETAS
balance_chaquetas = Equation(m, name="balance_chaquetas")
balance_chaquetas[...] = x[2] + s[2] == 750

# Balance de demanda de PANTALONES
balance_pantalones = Equation(m, name="balance_pantalones")
balance_pantalones[...] = x[3] + s[3] == 600

# Balance de demanda de GUANTES
balance_guantes = Equation(m, name="balance_guantes")
balance_guantes[...] = x[4] + s[4] == 500

# Paso 6: Crear lista de todas las ecuaciones
todas_las_restricciones = [
    restriccion_corte,
    restriccion_aislamiento,
    restriccion_costura,
    restriccion_empaque,
    balance_abrigos,
    balance_chaquetas,
    balance_pantalones,
    balance_guantes
]

modelo = Model(
    m,
    name="ProduccionExplicita",
    equations=todas_las_restricciones,
    sense=Sense.MAX,
    problem="LP",
    objective=objetivo
)

print("\n🔄 Resolviendo modelo...")
modelo.solve()


🔄 Resolviendo modelo...


,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,64625.0,9,9,LP,CPLEX,0.001
